In [1]:
import os

In [2]:
%pwd

'c:\\Users\\xaris\\Desktop\\SQL\\Teen Phone Addiction Project\\Teen_Phone_Addiction\\research'

In [3]:
os.chdir(r"C:\Users\xaris\Desktop\SQL\Teen Phone Addiction Project\\Teen_Phone_Addiction")

In [4]:
%pwd

'C:\\Users\\xaris\\Desktop\\SQL\\Teen Phone Addiction Project\\Teen_Phone_Addiction'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    dataset_download_url: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from Teen_Phone_Addiction.constants import *
from Teen_Phone_Addiction.utils.common import read_yaml, create_directories 

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH): 

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            dataset_download_url=config.dataset_download_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from Teen_Phone_Addiction import logger
from Teen_Phone_Addiction.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_dataset(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.dataset_download_url,
                filename=self.config.local_data_file
            )
            logger.info(f"Dataset downloaded: {filename}")
        else:
            logger.info("Dataset already downloaded.")
    
    def extract_zip_file(self):
        """
        Extracts the zip file to the specified directory.
        """
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)  

In [14]:
# Start the Pipeline
try:
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_dataset()
        data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-05 12:13:23,934: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-05 12:13:23,938: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-05 12:13:23,940: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-05 12:13:23,943: INFO: common: created directory at: artifacts]
[2025-08-05 12:13:23,944: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-05 12:13:28,270: INFO: 887910524: Dataset downloaded: artifacts/data_ingestion/teen_phone_addiction_dataset.zip]
